In [ ]:
from modeling_data import ModelingData
from lendingclub import LendingClub
from config import Config
from scorer import Scorer
from model import Model
from copy import copy

lc = LendingClub(Config('config.ini'))

lc.build_historic_data(data_split_type='stratified', historic_data_dir='../historical_data/')

train_md_roi = ModelingData(data=lc.train_data, target='roi')
train_md_roi.build()
train_md_roi.remove_outliers()

train_md_co = ModelingData(data=lc.train_data, target='charge_off')
train_md_co.build()
train_md_co.remove_outliers()

In [ ]:
charge_off_model = Model(train_md_co)
charge_off_model.tune_hyperparams(n_calls=20)
charge_off_model.fit()
charge_off_model.save(model_name='charge_off_model')

In [ ]:
roi_model = Model(train_md_roi)
roi_model.tune_hyperparams(n_calls=20)
roi_model.fit()
roi_model.save(model_name='roi_model')

In [ ]:
validation_md_roi = ModelingData(lc.validation_data, target='roi')
validation_md_roi.build()

validation_md_co = ModelingData(lc.validation_data, target='charge_off')
validation_md_co.build()

In [ ]:
roi_scorer = Scorer(model='saved_models/roi_model.pkl', data=validation_md_roi)
roi_scorer.score()

charge_off_scorer = Scorer(model='saved_models/charge_off_model.pkl', data=validation_md_co)
charge_off_scorer.score()

In [ ]:
test = copy(charge_off_scorer.data.modeling_data)
test['roi_actual'] = roi_scorer.data.target
test['roi_pred'] = roi_scorer.preds
test['charge_off_actual'] = charge_off_scorer.data.target
test['charge_off_pred'] = charge_off_scorer.preds

In [ ]:
buy = test.loc[(thing.roi_pred > 0.19) & (thing.charge_off_pred < 0.02)]
buy = pd.merge(lc.master_df[['id', 'total_pymnt']], buy, on='id')

In [ ]:
print('Model Selection Net Return: ' + str(np.sum(buy.total_pymnt) - np.sum(buy.funded_amnt)))
print('Model Selection ROI: ' + str((np.sum(buy.total_pymnt) - np.sum(buy.funded_amnt)) / np.sum(buy.funded_amnt)))

print('Number of notes invested:', str(len(buy.index)))
print('Perent of notes invested:', str(len(buy.index) / len(thing.index) * 100))